In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

In [13]:
driver = webdriver.Chrome()
driver.get("https://www.zomato.com/coimbatore/delivery")

scroll_pause_time = 3  
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '/order')]")))

elements = driver.find_elements(By.XPATH, "//a[contains(@href, '/order')]")

restaurant_links = [element.get_attribute('href') for element in elements]
restaurant_links = restaurant_links[:len(restaurant_links):2]
print(len(restaurant_links))

774


In [14]:
def scrape_page(url, headers):
    try:
        with requests.Session() as session:
            response = session.get(url, headers=headers)
            if response.status_code == 200:
                return response.content
            else:
                print("Failed to retrieve the page. Status code:", response.status_code)
                return None
    except Exception as e:
        print("An error occurred:", e)
        return None

In [15]:
areas = []

for row in restaurant_links:
        restaurant_link = row
        print(restaurant_link)
        try:
            driver = webdriver.Chrome()
            driver.get(restaurant_link)
            headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36' }
            content = scrape_page(restaurant_link, headers)
            soup = BeautifulSoup(content, 'html.parser')
            name = soup.find('h1',class_="sc-7kepeu-0 sc-iSDuPN fwzNdh").text.strip()

            cuisine_s = soup.find('div', class_="sc-fgfRvd gBMRZZ").find_all('span')
            cuisine = [span.text.strip() for span in cuisine_s[::2]]
            cuisine = [cos[:-1] for cos in cuisine]
            cuisine.append(cuisine_s[-1].text.strip())
            cuisine = ', '.join(map(str, cuisine))

            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.XPATH, "//a[@class='sc-clNaTc vNCcy']")))
            area_element = driver.find_element(By.XPATH, "//a[@class='sc-clNaTc vNCcy']")
            area = area_element.text.split(',')[-2]

            sub = restaurant_link[:-6]
            sub_driver = webdriver.Chrome()
            sub_driver.get(sub)

            address=sub_driver.find_elements(By.XPATH,"//p[contains(@class,'clKRrC')]")
            addresstext=[a1.text for a1 in address]
            addresstext=str(addresstext[0])

            for_call_loc = sub_driver.find_elements(By.XPATH, "//p[contains(@class, 'fanwIZ')]")
            calls = [element.text for element in for_call_loc]
            if len(calls) == 1:
                calls=str(calls[0])
                calls="'"+ calls
            else:
                calls=', '.join(map(str, calls))


            driver.quit()

        except Exception as e:
            print(f"Error processing restaurant link: {restaurant_link}, Error: {str(e)}")
            try:
                driver.quit()
            except:
                pass
            continue 
        
        areas.append({        
            'Restaurant URL': restaurant_link,
            'Name' : name,
            'Address': addresstext,
            'Cuisine' : cuisine, 
            'Area': area,
            'Contact Number' : calls
        })

areas_df = pd.DataFrame(areas)

# grouped = areas_df.groupby('Area')

# for area, group in grouped:
#     area_filename = f'zomato_links_{name}_{area}.csv'
#     group.to_csv(area_filename, index=False)
#     print(f"Links for area '{area}' have been saved to {area_filename}")

https://www.zomato.com/coimbatore/ss-hyderabad-biryani-coimbatore-gandhipuram/order
https://www.zomato.com/coimbatore/a1-chips-since-1970-2-gandhipuram/order
https://www.zomato.com/coimbatore/hotel-chola-1-gandhipuram/order
https://www.zomato.com/coimbatore/geetha-canteen-gandhipuram/order
https://www.zomato.com/coimbatore/sree-subbu-mess-race-course/order
https://www.zomato.com/coimbatore/hmr-grand-kitchen-gandhipuram/order
https://www.zomato.com/coimbatore/hotel-cheap-and-best-town-hall/order
https://www.zomato.com/coimbatore/abitha-biriyani-hotel-ukkadam/order
https://www.zomato.com/coimbatore/kfc-1-gandhipuram/order
https://www.zomato.com/coimbatore/salem-rr-biryani-gandhipuram/order
https://www.zomato.com/coimbatore/thalassery-restaurant-1-ukkadam/order
Error processing restaurant link: https://www.zomato.com/coimbatore/thalassery-restaurant-1-ukkadam/order, Error: list index out of range
https://www.zomato.com/coimbatore/a2b-adyar-ananda-bhavan-gandhipuram/order
https://www.zomat

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Error processing restaurant link: https://www.zomato.com/coimbatore/thepan-tiffin-stall-gandhipuram/order, Error: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=125.0.6422.77)
Stacktrace:
	GetHandleVerifier [0x00007FF60A331F22+60322]
	(No symbol) [0x00007FF60A2ACE99]
	(No symbol) [0x00007FF60A167EBA]
	(No symbol) [0x00007FF60A15FEA2]
	(No symbol) [0x00007FF60A1509C4]
	(No symbol) [0x00007FF60A152212]
	(No symbol) [0x00007FF60A150CF1]
	(No symbol) [0x00007FF60A1504D1]
	(No symbol) [0x00007FF60A150410]
	(No symbol) [0x00007FF60A14E39D]
	(No symbol) [0x00007FF60A14E9DC]
	(No symbol) [0x00007FF60A16AC91]
	(No symbol) [0x00007FF60A1FC5FE]
	(No symbol) [0x00007FF60A1DC21A]
	(No symbol) [0x00007FF60A1FBC80]
	(No symbol) [0x00007FF60A1DBFC3]
	(No symbol) [0x00007FF60A1A9617]
	(No symbol) [0x00007FF60A1AA211]
	GetHandleVerifier [0x00007FF60A64946D+3301613]
	GetHandleVerifier [0x00007FF60A693693+3605267]
	GetHandleVerifier [0x00007FF60A689410+3563664]
	GetHandleVe

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Error processing restaurant link: https://www.zomato.com/coimbatore/foodie-fort-ukkadam/order, Error: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=125.0.6422.77)
Stacktrace:
	GetHandleVerifier [0x00007FF60A331F22+60322]
	(No symbol) [0x00007FF60A2ACE99]
	(No symbol) [0x00007FF60A167EBA]
	(No symbol) [0x00007FF60A15FEA2]
	(No symbol) [0x00007FF60A1509C4]
	(No symbol) [0x00007FF60A152212]
	(No symbol) [0x00007FF60A150CF1]
	(No symbol) [0x00007FF60A1504D1]
	(No symbol) [0x00007FF60A150410]
	(No symbol) [0x00007FF60A14E39D]
	(No symbol) [0x00007FF60A14E9DC]
	(No symbol) [0x00007FF60A16AC91]
	(No symbol) [0x00007FF60A1FC5FE]
	(No symbol) [0x00007FF60A1DC21A]
	(No symbol) [0x00007FF60A1FBC80]
	(No symbol) [0x00007FF60A1DBFC3]
	(No symbol) [0x00007FF60A1A9617]
	(No symbol) [0x00007FF60A1AA211]
	GetHandleVerifier [0x00007FF60A64946D+3301613]
	GetHandleVerifier [0x00007FF60A693693+3605267]
	GetHandleVerifier [0x00007FF60A689410+3563664]
	GetHandleVerifier [0x00

...

Error processing restaurant link: https://www.zomato.com/coimbatore/magical-multi-cuisine-restaurant-peelamedu/order, Error: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=125.0.6422.77)
Stacktrace:
	GetHandleVerifier [0x00007FF60A331F22+60322]
	(No symbol) [0x00007FF60A2ACE99]
	(No symbol) [0x00007FF60A167EBA]
	(No symbol) [0x00007FF60A15FEA2]
	(No symbol) [0x00007FF60A1509C4]
	(No symbol) [0x00007FF60A152212]
	(No symbol) [0x00007FF60A150CF1]
	(No symbol) [0x00007FF60A1504D1]
	(No symbol) [0x00007FF60A150410]
	(No symbol) [0x00007FF60A14E39D]
	(No symbol) [0x00007FF60A14E9DC]
	(No symbol) [0x00007FF60A16AC91]
	(No symbol) [0x00007FF60A1FC5FE]
	(No symbol) [0x00007FF60A1DC21A]
	(No symbol) [0x00007FF60A1FBC80]
	(No symbol) [0x00007FF60A1DBFC3]
	(No symbol) [0x00007FF60A1A9617]
	(No symbol) [0x00007FF60A1AA211]
	GetHandleVerifier [0x00007FF60A64946D+3301613]
	GetHandleVerifier [0x00007FF60A693693+3605267]
	GetHandleVerifier [0x00007FF60A689410+3563664]
	

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Error processing restaurant link: https://www.zomato.com/coimbatore/rocket-joes-pizza-peelamedu/order, Error: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=125.0.6422.77)
Stacktrace:
	GetHandleVerifier [0x00007FF60A331F22+60322]
	(No symbol) [0x00007FF60A2ACE99]
	(No symbol) [0x00007FF60A167EBA]
	(No symbol) [0x00007FF60A15FEA2]
	(No symbol) [0x00007FF60A1509C4]
	(No symbol) [0x00007FF60A152212]
	(No symbol) [0x00007FF60A150CF1]
	(No symbol) [0x00007FF60A1504D1]
	(No symbol) [0x00007FF60A150410]
	(No symbol) [0x00007FF60A14E39D]
	(No symbol) [0x00007FF60A14E9DC]
	(No symbol) [0x00007FF60A16AC91]
	(No symbol) [0x00007FF60A1FC5FE]
	(No symbol) [0x00007FF60A1DC21A]
	(No symbol) [0x00007FF60A1FBC80]
	(No symbol) [0x00007FF60A1DBFC3]
	(No symbol) [0x00007FF60A1A9617]
	(No symbol) [0x00007FF60A1AA211]
	GetHandleVerifier [0x00007FF60A64946D+3301613]
	GetHandleVerifier [0x00007FF60A693693+3605267]
	GetHandleVerifier [0x00007FF60A689410+3563664]
	GetHandleVerifi

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Error processing restaurant link: https://www.zomato.com/coimbatore/hotel-sathar-rowther-hsr-rs-puram/order, Error: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=125.0.6422.77)
Stacktrace:
	GetHandleVerifier [0x00007FF60A331F22+60322]
	(No symbol) [0x00007FF60A2ACE99]
	(No symbol) [0x00007FF60A167EBA]
	(No symbol) [0x00007FF60A15FEA2]
	(No symbol) [0x00007FF60A1509C4]
	(No symbol) [0x00007FF60A152212]
	(No symbol) [0x00007FF60A150CF1]
	(No symbol) [0x00007FF60A1504D1]
	(No symbol) [0x00007FF60A150410]
	(No symbol) [0x00007FF60A14E39D]
	(No symbol) [0x00007FF60A14E9DC]
	(No symbol) [0x00007FF60A16AC91]
	(No symbol) [0x00007FF60A1FC5FE]
	(No symbol) [0x00007FF60A1DC21A]
	(No symbol) [0x00007FF60A1FBC80]
	(No symbol) [0x00007FF60A1DBFC3]
	(No symbol) [0x00007FF60A1A9617]
	(No symbol) [0x00007FF60A1AA211]
	GetHandleVerifier [0x00007FF60A64946D+3301613]
	GetHandleVerifier [0x00007FF60A693693+3605267]
	GetHandleVerifier [0x00007FF60A689410+3563664]
	GetHandle

In [16]:
areas_df.to_excel('Zomato_Coimbatore_2.xlsx',index=False)
areas_df.to_csv('Zomato_Coimbatore_2.csv',index=False)

In [5]:
import pandas as pd
restaurant_links = restaurant_links[:150]
data=[]
for link in restaurant_links:
    try :
        print(link)
        headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36' }
        content = scrape_page(link, headers)
        if content:
                soup = BeautifulSoup(content, 'html.parser')
                name = soup.find('h1',class_="sc-7kepeu-0 sc-iSDuPN fwzNdh").text.strip()
                # print(name)
                cousine_s = soup.find('div', class_="sc-fgfRvd gBMRZZ").find_all('span')
                cousine = [span.text.strip() for span in cousine_s[::2]]
                cousine = [cos[:-1] for cos in cousine]
                cousine.append(cousine_s[-1].text.strip())
                cousine = ', '.join(map(str, cousine))
                # print(cousine)
                sub = link[:-6]
                sub_driver = webdriver.Chrome()
                sub_driver.get(sub)
                for_call_loc = sub_driver.find_elements(By.XPATH, "//p[contains(@class, 'fanwIZ')]")
                calls = [element.text for element in for_call_loc]
                if len(calls) == 1:
                    calls=str(calls[0])
                    calls="'"+ calls
                else:
                    calls=', '.join(map(str, calls))
                # print(calls)
                area=sub_driver.find_elements(By.XPATH,"//a[contains(@class,'vNCcy')]")
                areatext=[a.text for a in area]
                areatext=str(areatext[0])
                # print(areatext)
                address=sub_driver.find_elements(By.XPATH,"//p[contains(@class,'clKRrC')]")
                addresstext=[a1.text for a1 in address]
                addresstext=str(addresstext[0])
                # print(addresstext)
                time=sub_driver.find_elements(By.XPATH,"//span[contains(@class,'sc-kasBVs dfwCXs')]")
                timetext=[t.text for t in time]
                timetext=str(timetext[0])
                # print(timetext)
                main_driver = webdriver.Chrome()
                main_driver.get(link)
                dil_rat_s = main_driver.find_elements(By.XPATH,"//div[contains(@class,'sc-1q7bklc-1 cILgox')]")
                dil_rat = [s.text for s in dil_rat_s]
                print(dil_rat[1])
                catgeory = main_driver.find_elements(By.XPATH,"//h4[contains(@class,'sc-1hp8d8a-0 sc-kUQWMX bmNHuj')]")
                cattext=[c.text for c in catgeory]
                print(cattext)
                for cat in cattext :
                    #  print(cat)
                    h4_element = main_driver.find_element(By.XPATH, f"//h4[text()='{cat}']")
                    section = h4_element.find_element(By.XPATH, "./ancestor-or-self::section[1]")
                    #  print(section.get_attribute('class'))
                    #  print(section.get_attribute('outerHTML'))
                    cards = section.find_elements(By.XPATH, ".//div[@class='sc-1s0saks-17 bGrnCu']")
                    #  print(len(cards))
                    #  cards = cards[:2]
                    for card in cards :
                        # print(card.get_attribute('outerHTML'))
                        food_s = card.find_element(By.XPATH,".//h4[contains(@class,'sc-1s0saks-15 iSmBPS')]")
                        food_name = food_s.text
                        print(food_name)
                        price_s = card.find_element(By.XPATH,".//span[contains(@class,'sc-17hyc2s-1 cCiQWA')]")
                        price = price_s.text
                        price=price[1:]
                        price=int(price)
                        print(price)
                        p_s = card.find_element(By.XPATH,".//p[contains(@class,'sc-1s0saks-12 hcROsL')]")
                        p = p_s.text
                        print(p)
                        data.append([link, name, addresstext, areatext, cousine, calls, timetext[:-8], cat, food_name, price, p])
                        #   img_url_s = card.find_element(By.XPATH, ".//img[1]")
                        #   print(img_url_s)
                        #   img_url = img_url_s.get_attribute('outerHTML')
                        #   print(img_url)
                #       print(section) 
                # print(str(catgeory.text))
                    #  catstr=', '.join(map(str, cattext))
                    #  print(catstr)
                # imag = section.find_elements(By.XPATH,"//img[contains(@src,'/data')]")
                # print(imag[0].get_attribute('src'))
                # cat = soup.find('section',class_="sc-ldcLGC gNlnSZ")
                # print(cat)
                data.append([link,name,addresstext,areatext,cousine,calls,timetext[:-8],dil_rat[1]])
                sub_driver.quit()
                main_driver.quit()
    except Exception as e:
        print(f"Error processing link: {link}, Error: {str(e)}")
        try:
            driver.quit()
        except:
            pass
        continue  


df = pd.DataFrame(data, columns=['Restaurant URL', 'Name','Address','Area','Cuisine', 'Contact Number','Restraurant Timings','Category','Item','Item Price','Item Discription'])
df.to_csv('zomato_Coimbatore.csv', index=False)
# print("Data has been scraped and written to zomato_vesu.csv \n")

https://www.zomato.com/coimbatore/ravuthar-biriyani-hotel-thudiyalur/order
4.1
['Parotta', 'Parotta Combo', 'Rice And Biryani', 'Summa Semma Combos', 'Meal For One', 'Half Bucket Biriyani’s', 'Bucket Biriyanis', 'Couples Pack', 'Chicken Biriyani Combos', 'Beef Biriyani Combos', 'Mutton Biriyani Combos', 'Empty Biriyani Combos', 'Grill', 'Mayonnaise', 'Biriyanis', 'Starters', 'Boneless Chicken Drys', 'Boneless Chicken Gravys', 'Beef Drys', 'Beef Gravys', 'Mutton Drys', 'Mutton Gravys', 'Lollipop Drys', 'Fish Boneless Drys', 'Fish Boneless Gravys', 'Prawn Drys', 'Prawn Gravys', 'Egg Masalas', 'Fried Rice', 'Noodles', 'Party Combos']
Ravuthars Special Parotta
45

Egg Kothu Parotta
218

Chicken Kothu Parotta
285

Beef Kothu Parotta
285

Veechu Parotta
53

Egg Veechu Parotta
78

Egg Lappa
255

Chicken Lappa
270

Beef Lappa
270

2Xparotta With Pepper Chicken
285

2Xparotta With Butter Chicken
285

2Xparotta With Beef Chukka
285

2Xparotta With Kerala Beef
305

2Xparotta With Mutton Chukka
37

In [6]:
df.to_excel('Zomato_Coimbatore.xlsx',index=False)

In [43]:
import pandas as pd
restaurant_links = ['https://www.zomato.com/surat/la-pinoz-pizza-vesu/order']
data=[]
for link in restaurant_links:
    print(link)
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36' }
    content = scrape_page(link, headers)
    if content:
            soup = BeautifulSoup(content, 'html.parser')
            name = soup.find('h1',class_="sc-7kepeu-0 sc-iSDuPN fwzNdh").text.strip()
            # print(name)
            cousine_s = soup.find('div', class_="sc-fgfRvd gBMRZZ").find_all('span')
            cousine = [span.text.strip() for span in cousine_s[::2]]
            cousine = [cos[:-1] for cos in cousine]
            cousine.append(cousine_s[-1].text.strip())
            cousine = ', '.join(map(str, cousine))
            # print(cousine)
            sub = link[:-6]
            sub_driver = webdriver.Chrome()
            sub_driver.get(sub)
            for_call_loc = sub_driver.find_elements(By.XPATH, "//p[contains(@class, 'fanwIZ')]")
            calls = [element.text for element in for_call_loc]
            if len(calls) == 1:
                calls=str(calls[0])
                calls="'"+ calls
            else:
                calls=', '.join(map(str, calls))
            # print(calls)
            area=sub_driver.find_elements(By.XPATH,"//a[contains(@class,'vNCcy')]")
            areatext=[a.text for a in area]
            areatext=str(areatext[0])
            # print(areatext)
            address=sub_driver.find_elements(By.XPATH,"//p[contains(@class,'clKRrC')]")
            addresstext=[a1.text for a1 in address]
            addresstext=str(addresstext[0])
            # print(addresstext)
            time=sub_driver.find_elements(By.XPATH,"//span[contains(@class,'sc-kasBVs dfwCXs')]")
            timetext=[t.text for t in time]
            timetext=str(timetext[0])
            # print(timetext)
            main_driver = webdriver.Chrome()
            main_driver.get(link)
            dil_rat_s = main_driver.find_elements(By.XPATH,"//div[contains(@class,'sc-1q7bklc-1 cILgox')]")
            dil_rat = [s.text for s in dil_rat_s]
            print(dil_rat[1])
            catgeory = main_driver.find_elements(By.XPATH,"//h4[contains(@class,'sc-1hp8d8a-0 sc-kUQWMX bmNHuj')]")
            cattext=[c.text for c in catgeory]
            print(cattext)
            for cat in cattext :
                #  print(cat)
                 h4_element = main_driver.find_element(By.XPATH, f"//h4[text()='{cat}']")
                 section = h4_element.find_element(By.XPATH, "./ancestor-or-self::section[1]")
                #  print(section.get_attribute('class'))
                #  print(section.get_attribute('outerHTML'))
                 cards = section.find_elements(By.XPATH, ".//div[@class='sc-1s0saks-17 bGrnCu']")
                #  print(len(cards))
                #  cards = cards[:1]
                 for card in cards :
                    # print(card.get_attribute('outerHTML'))
                    food_s = card.find_element(By.XPATH,".//h4[contains(@class,'sc-1s0saks-15 iSmBPS')]")
                    food_name = food_s.text
                    print(food_name)
                    price_s = card.find_element(By.XPATH,".//span[contains(@class,'sc-17hyc2s-1 cCiQWA')]")
                    price = price_s.text
                    price=price[1:]
                    price=int(price)
                    print(price)
                    p_s = card.find_element(By.XPATH,".//p[contains(@class,'sc-1s0saks-12 hcROsL')]")
                    p = p_s.text
                    print(p)
                    veg_s = card.find_element(By.XPATH,"//div[contains(@class,'sc-1tx3445-0 kcsImg sc-1s0saks')]")
                    print(veg_s.get_attribute('type'))
                    if veg_s.get_attribute('type') == 'veg':
                        veg = 1
                    else:
                         veg = 0
                    # veg =1
                    alco = 0
                    data.append([ cat, food_name, veg, alco, p, price])
                    #   img_url_s = card.find_element(By.XPATH, ".//img[1]")
                    #   print(img_url_s)
                    #   img_url = img_url_s.get_attribute('outerHTML')
                    #   print(img_url)

                      
            sub_driver.quit()
            main_driver.quit()
            # data.append([link,name,addresstext,areatext,cousine,calls,timetext[:-8],dil_rat[1]])
            #       print(section) 
            # print(str(catgeory.text))
                #  catstr=', '.join(map(str, cattext))
            #     #  print(catstr)
            # imag = section.find_elements(By.XPATH,"//img[contains(@src,'/data')]")
            # print(imag[0].get_attribute('src'))
            # cat = soup.find('section',class_="sc-ldcLGC gNlnSZ")
            # print(cat)

df = pd.DataFrame(data, columns=['Category','Item_Name','Is_Veg(1:veg,0:non-veg)','Is_Alcohol(1:alcoholic,0:non-alcoholic)','Discription','Price'])
df.to_csv('zomato_vesu.csv', index=False)
print("Data has been scraped and written to zomato_vesu.csv \n")

https://www.zomato.com/surat/la-pinoz-pizza-vesu/order
4.1
['Half N Half Pizza', 'Personal Slice Pizza', 'Veg Pizza', 'Classic Pizzas For Classic Maniacs', 'Sides', 'Match Day Combos', 'Dessert & Beverages']
Veg Half N Half Pizza Large ( 33 Cm)
795

veg
Cheesy Macaroni (personal Giant Slice (22.5 Cm))
286
Paneer, Onions, Capsicum, Olives, Baby Corn With Loads Of Cheese
veg
Margherita Pizza (personal Giant Slice (22.5 Cm))
134
A classic cheesy Margherita. Cant go wrong. [Fat-14.3 per 100 g, Protein-12.6 per ... read more
veg
Paneer 65 (Personal Giant Slice (22.5 Cm))
263
A Combination Of Onion, Capsicum, Red Paprika, Paneer 65 With Extra Cheese
veg
Spicy Veggie 65 (personal Giant Slice (22.5 Cm))
286
Red Peprika,Jalapenos,Capsicum,Onion,Paneer 65 With Extra Cheese
veg
Mac Special (personal Giant Slice (22.5 Cm))
196
Mac Special
veg
Makhani Do Pyaza (personal Giant Slice (22.5 Cm))
134
Makhani Gravy, Onion & Cheese
veg
Onion Twist (personal Giant Slice (22.5 Cm))
134
Twisted combination 

In [ ]:
import pandas as pd

# df = pd.read_csv('zomato_vesu.csv')
# restaurant_links = df['Restaurant URL'].iloc[46:54]
for link in restaurant_links:
    try :
        print(link)
        headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36' }
        content = scrape_page(link, headers)
        if content:
                soup = BeautifulSoup(content, 'html.parser')
                name = soup.find('h1',class_="sc-7kepeu-0 sc-iSDuPN fwzNdh").text.strip()
                file_name = 'Zomato_'+name+'.csv'
                
                main_driver = webdriver.Chrome()
                main_driver.get(link)

                catgeory = main_driver.find_elements(By.XPATH,"//h4[contains(@class,'sc-1hp8d8a-0 sc-kUQWMX bmNHuj')]")
                cattext=[c.text for c in catgeory]
                print(cattext)
                data=[]
                for cat in cattext :
                    h4_element = main_driver.find_element(By.XPATH, f"//h4[text()='{cat}']")
                    section = h4_element.find_element(By.XPATH, "./ancestor-or-self::section[1]")

                    cards = section.find_elements(By.XPATH, ".//div[@class='sc-1s0saks-17 bGrnCu']")

                    for card in cards :
                        food_s = card.find_element(By.XPATH,".//h4[contains(@class,'sc-1s0saks-15 iSmBPS')]")
                        food_name = food_s.text
                        print(food_name)
                        price_s = card.find_element(By.XPATH,".//span[contains(@class,'sc-17hyc2s-1 cCiQWA')]")
                        price = price_s.text
                        price=price[1:]
                        price=float(price)
                        print(price)
                        p_s = card.find_element(By.XPATH,".//p[contains(@class,'sc-1s0saks-12 hcROsL')]")
                        p = p_s.text
                        print(p)
                        veg_s = card.find_element(By.XPATH,"//div[contains(@class,'sc-1tx3445-0 kcsImg sc-1s0saks')]")
                        print(veg_s.get_attribute('type'))
                        if veg_s.get_attribute('type') == 'veg':
                            veg = 1
                        else:
                            veg = 0
                        alco = 0
                        data.append([ cat, food_name, veg, alco, p, price])
                        
                        df = pd.DataFrame(data, columns=['Category','Item_Name','Is_Veg(1:veg,0:non-veg)','Is_Alcohol(1:alcoholic,0:non-alcoholic)','Description','Price'])
                        df.to_csv(file_name, index=False)
                        print("Data has been scraped and written to "+file_name)
                # sub_driver.quit()
                main_driver.quit()
    except Exception as e:
        print(f"Error processing link: {link}, Error: {str(e)}")
        try:
            driver.quit()
        except:
            pass
        continue  



https://www.zomato.com/coimbatore/kfc-ganapathy/order
['Burgers', 'All Day Lunch Special Meal Box', 'Match Day Combos (Save upto 35%)', 'Peri Peri Chicken Strips & Leg Pc (up to 25%)', 'Buckets/ Meals for 1-2 (up to 15% off)', 'Buckets/ Meals for 3-4 (up to 32% off)', 'Chicken Rolls', 'Value Snackers (Save upto 25%)', 'Hot & Crispy Chicken, Wings', 'Boneless Chicken Popcorn', 'Grilled Smoky Red Chicken', 'Sides and Dips', 'Desserts & Beverages']
Paneer Zinger Burger
208.57
It's Back! Crunchy Indian Paneer Patty Burger with tangy sauce and Veggies
veg
Data has been scraped and written to Zomato_KFC.csv
Spicy Zinger Burger and Popcorn Meal
439.05
Caribbean Spicy Zinger Burger with Medium Popcorn and Pepsi PET 475ml at a ... read more
veg
Data has been scraped and written to Zomato_KFC.csv
Spicy Zinger Burger
219.05
New Caribbean Spicy Zinger Burger with Spicy Bun & Spicy chilli sauce with ... read more
veg
Data has been scraped and written to Zomato_KFC.csv
Spicy Zinger Burger and Chicke

: 

: 

In [ ]:
# <img alt="Margherita Pizza (personal Giant Slice (22.5 Cm))" src="https://b.zmtcdn.com/data/dish_photos/92b/a413a537c40ab2d7b7e03bac6baea92b.jpeg?output-format=webp&amp;fit=around|130:130&amp;crop=130:130;*,*" loading="lazy" class="sc-s1isp7-5 fyZwWD">

In [ ]:
# # Close the driver                    sc-s1isp7-5 fyZwWD
# driver.quit()